## 张量常用的数据统计

比如统计最值，最值的位置，均值，范数等信息。

### 向量范数
向量范数(Vector Norm)表征向量"长度"的一种度量方法。常见的范数有：L1,L2,无穷大范数：例如向量的范数定义
- L1 范数：向量所有元素的绝对值之和
- L2 范数：向量所有元素的平方和，在开根号
- inf 范数：向量所以元素绝对值的最大值

利用tf.norm(x,ord)来求解：
- ord =1 ：表示L1
- ord = 2: L2 
- ord = np.inf 表示最大值

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.3.0


In [6]:
x = tf.ones([2,2])
print(tf.norm(x,ord=1))
print(tf.norm(x,ord=2))
print(tf.norm(x,ord=np.inf))

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [12]:
#指定轴上范数
x = tf.ones([2,3])
# 行的方向
print(tf.norm(x,ord=1,axis=0))
print(tf.norm(x,ord=2,axis=0))
# 列的方向
print(tf.norm(x,ord=2,axis=1))

tf.Tensor([2. 2. 2.], shape=(3,), dtype=float32)
tf.Tensor([1.4142135 1.4142135 1.4142135], shape=(3,), dtype=float32)
tf.Tensor([1.7320508 1.7320508], shape=(2,), dtype=float32)


### 查找最值，均值，和以及最值的索引号
- tf.reduce_max
- tf.reduce_min
- tf.reduce_mean
- tf.reduce_sum
- tf.argmax
- tf.argmin

>参数：
- 第一个参数input_tensor： 输入的待降维的tensor;
- 第二个参数axis： 指定的轴方向，如果不指定，则计算所有元素;
- 第三个参数keep_dims：是否降维度，设置为True，输出的结果保持输入tensor的形状，设置为False，输出结果会降低维度;


例如 shape[4,10], 记录的时候样本个数为4，10个类别的概率，需要找出每个样本的最大概率的值。

In [19]:
x = tf.random.uniform(shape=[4,10],minval=0,maxval=1,dtype=tf.float32)
# 关于指定轴的理解：这里要求每一行中列元素的最大值，所以设置为列的方向
print(tf.reduce_max(x,axis=1))
print(tf.reduce_min(x,axis=1))
print(tf.reduce_mean(x,axis=1))
print(tf.reduce_sum(x,axis=1))

tf.Tensor([0.90580153 0.935712   0.9590367  0.94939566], shape=(4,), dtype=float32)
tf.Tensor([0.12353003 0.06850481 0.03555584 0.26242328], shape=(4,), dtype=float32)
tf.Tensor([0.5966229 0.5580625 0.508815  0.4725522], shape=(4,), dtype=float32)
tf.Tensor([5.966229 5.580625 5.08815  4.725522], shape=(4,), dtype=float32)


In [20]:
# 不指定轴，则计算所有的元素的最值
x = tf.random.uniform(shape=[4,10],minval=0,maxval=1,dtype=tf.float32)
tf.reduce_max(x),tf.reduce_min(x),tf.reduce_mean(x),tf.reduce_sum(x)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.99992895>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.020911574>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.5184097>,
 <tf.Tensor: shape=(), dtype=float32, numpy=20.73639>)

一个实际的计算关于损失值的过程，就需要使用上面的那些API来计算。

In [26]:
out = tf.random.normal([4,10])
y = tf.constant([1,2,2,0])
y = tf.one_hot(y,depth=10)
# 计算每个样本的误差：均方差
loss = tf.keras.losses.mse(y,out)
mse = tf.reduce_mean(tf.square(out-y),axis=1)
print(mse)
print("样本误差：",loss)
# 计算总的评价误差
loss = tf.reduce_mean(loss)
print("总的均值误差：",loss)

tf.Tensor([2.7964349 0.8372906 1.47877   1.5083137], shape=(4,), dtype=float32)
样本误差： tf.Tensor([2.7964349 0.8372906 1.47877   1.5083137], shape=(4,), dtype=float32)
总的均值误差： tf.Tensor(1.6552023, shape=(), dtype=float32)


### 最值索引号的获取

-  tf.argmax(x, axis)
-  tf.argmin(x, axis)


In [5]:
out = tf.random.normal([2,10])
# 转换成概率值
out = tf.nn.softmax(out,axis=1)
print(out)

tf.Tensor(
[[0.31100145 0.10704491 0.02488793 0.01039608 0.18288529 0.11658079
  0.03200262 0.06312642 0.06114587 0.09092861]
 [0.2951885  0.03986979 0.05252219 0.0377373  0.1492606  0.04129728
  0.12838507 0.08179147 0.15033846 0.02360938]], shape=(2, 10), dtype=float32)


In [6]:
# 获得最大值的索引的位置
pred = tf.argmax(out,axis=1)
print(pred)


tf.Tensor([0 0], shape=(2,), dtype=int64)


### tensor的比较

可以想象一个应用场景，就是用来统计分类任务的准确率的时候，需要将预测值与真实标签进行比较。
- tf.equal(a,b)
- tf.math.equal(a,b)

返回值是True和False

还有其他的比较函数
tf.math.greater  𝑎 > 𝑏  
tf.math.less  𝑎 < 𝑏  
tf.math.greater_equal  𝑎 ≥ 𝑏  
tf.math.less_equal  𝑎 ≤ 𝑏  
tf.math.not_equal  𝑎 ≠ 𝑏  
tf.math.is_nan  𝑎 = nan

In [8]:
# 模拟生成预测值
out = tf.random.normal([100,10])
out = tf.nn.softmax(out,axis=1)

pred = tf.argmax(out,axis=1)
print(pred)

tf.Tensor(
[5 7 2 3 5 5 0 6 0 6 0 9 4 9 3 2 0 0 8 0 6 9 6 9 8 3 9 9 5 0 6 7 0 3 4 7 4
 0 6 3 1 6 5 4 1 5 1 2 4 2 0 5 4 1 8 5 7 3 7 0 8 1 6 9 2 5 1 4 8 7 1 3 8 4
 6 1 5 4 0 3 3 8 4 6 5 8 0 6 0 5 7 6 0 9 5 1 7 9 5 9], shape=(100,), dtype=int64)


In [11]:
y = tf.random.uniform([100],dtype=tf.int64,maxval=10)
print(y)

tf.Tensor(
[1 3 4 2 6 6 8 8 9 9 7 5 5 2 3 4 2 1 1 1 1 5 5 0 5 3 2 7 9 9 2 1 2 1 0 2 7
 9 5 8 1 8 0 1 8 7 0 0 7 6 8 6 2 9 0 8 4 4 9 5 1 1 6 2 0 5 8 1 3 3 9 7 7 5
 8 4 9 5 8 2 8 3 6 8 2 8 6 8 0 0 9 3 2 6 4 8 9 7 4 1], shape=(100,), dtype=int64)


In [12]:
# 使用tf.equal来获得比较的结果，返回值是True和False
out = tf.equal(pred,y)
print(out)

tf.Tensor(
[False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False  True False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False  True  True False False  True False False False False False False
 False False False False False False False False False False False False
 False  True False False  True False False False False False False False
 False False False False], shape=(100,), dtype=bool)


In [14]:
out = tf.cast(out,dtype =tf.float32)
corret = tf.reduce_sum(out)/y.shape[0]
print(corret)

tf.Tensor(0.08, shape=(), dtype=float32)
